# The impact of the last match 

The analysis is made to see what is the chance of winning if the club won the previous match.

Analysis made on the basis of data from the "football" database.\
Notebook shows the usefulness of the collected data in the ETL process and presents an example of its usage.

### Import packeges

In [102]:
import pandas as pd
import numpy as np
import sys

sys.path.append('../')

from data.custom_functions import postgres_func as pf

import warnings
warnings.filterwarnings('ignore')

### Create connection and get data by sql

In [103]:
conn = pf.create_connection()
cur = conn.cursor()
cur.execute(
'''
    select 
        c.club_name as home_club
        ,d.club_name as away_club
        ,m.who_win
        ,matchday
    from football.match_stats m
    left join football.clubs c
    on m.home_id = c.club_id
    left join football.clubs d
    on m.away_id = d.club_id
''')

df = pd.DataFrame(cur.fetchall(), columns=['home_club', 'away_club', 'who_win', 'matchday'])
conn.close()


### Transform data and create statistics

In [106]:
df = df.drop(df[df['who_win'] == 'err'].index)
df_waw = pd.DataFrame(columns=('club', 'perc_waw'))
clubs = df['home_club'].unique()

for club in clubs:
    df_club = df[(df['home_club'] == club) | (df['away_club'] == club)]

    df_club.loc[(df['home_club'] == club) & (df['who_win'] == 'home'), 'win'] = 1
    df_club.loc[(df['away_club'] == club) & (df['who_win'] == 'away'), 'win'] = 1
    df_club['win'] = df_club['win'].fillna(0)

    df_club['win'] = df_club['win'].astype('int')
    df_club = df_club.sort_values(by='matchday')

    all_matches = len(df_club)
    win_after_win = 0

    for i, (_, row) in enumerate(df_club.iterrows()):
        if i == 0:
            last_row = row
            continue
        
        new_row = row

        if new_row['win'] == 1 and last_row['win'] == 1:
            win_after_win += 1
        
        last_row = row
    
    win_after_win_perc = round(100*win_after_win/all_matches, 2)

    df_waw = df_waw._append({'club': club, 'perc_waw': win_after_win_perc}, ignore_index=True)

df_waw = df_waw.sort_values(by='perc_waw', ascending=False)

### Results

In [107]:
df_waw # perc_waw -> winning percentage after winning

,club,perc_waw
0,Liverpool,47.13
15,Manchester City,46.82
10,Arsenal,31.43
9,Manchester United,22.73
7,Tottenham,22.41
13,Chelsea,19.54
8,Newcastle,17.24
5,Leicester,17.11
11,Aston Villa,14.29
1,West Ham,13.79


In [108]:
perc = df_waw['perc_waw'].mean()
print('Avarage chance of winning if club won last match:')
print(str(round(perc)) + '%')

Avarage chance of winning if club won last match:
14%
